# Data acquisition

The common analysis research question is based on one specific dataset. You should get the [Combined wildland fire datasets for the United States and certain territories, 1800s-Present (combined wildland fire polygons)](https://www.sciencebase.gov/catalog/item/61aa537dd34eb622f699df81) dataset. This dataset was collected and aggregated by the US Geological Survey. The dataset is relatively well documented. Fire polygons are available in ArcGIS and GeoJSON formats. 
We have been assigned one US city that will form the basis for your individual analysis. This can be found from [this Google spreadsheet](https://docs.google.com/spreadsheets/d/1cmTW5fgU3KyH6JbrRao-qWjzu2GovKk_BkA7a-poGFw/edit?usp=drive_link).

This notebook has dependencies on [Pyproj](https://pyproj4.github.io/pyproj/stable/index.html) and the [geojson](https://pypi.org/project/geojson/) module. Pyproj and geojson can be installed via pip. 

### License
This code example was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.0 - August 13, 2023

### Preliminaries
First we start with some imports and some constant definitions.

In [1]:
# Import some standard python modules

#
# The module pyproj is a standard module that provides tools to convert between different geodesic coordinate systems
# and for calculating distances between points (coordinates) in a specific geodesic system.
#
from pyproj import Transformer, Geod
import json
import geojson
import os
import pandas as pd
from tqdm import tqdm

# Ignore warning messages to suppress unnecessary output
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Define a constant variable for the name of the JSON file to extract data from
EXTRACT_FILENAME = "USGS_Wildland_Fire_Combined_Dataset.json"
# Construct the full path to the JSON file by joining the relative path "..\data" with the filename
FILENAME = os.path.join("..", "data", EXTRACT_FILENAME)
# Print the value of the FILENAME variable, indicating where the sample data is expected to be found
print(f"{FILENAME=}")
# Create a dictionary named CITY_LOCATION to store information about a city named 'tulare'
CITY_LOCATION = {
    'tulare': {
        'city': 'Tulare',
        'latlon': [36.2714, -118.7770]
    }
}

FILENAME='..\\data\\USGS_Wildland_Fire_Combined_Dataset.json'


## Load the wildfire data using the geojson module

We use the GeoJSON module ([documentation](https://pypi.org/project/geojson/), [GitHub repo](https://github.com/jazzband/geojson)) to load the file. This module works mostly the way one would expect. GeoJSON is mostly just JSON, so actually, we don't even really need to use the GeoJSON module. However, that module will do some conversion of Geo type things to something useful.

In [3]:
# Load the data from the opened file using the geojson library's 'load' function
print(f"Attempting to open '{FILENAME}'")
with open(FILENAME, "r") as f:
    gj_data = geojson.load(f)

Attempting to open '..\data\USGS_Wildland_Fire_Combined_Dataset.json'


In [4]:
# Get a list of keys from the 'gj_data' object
gj_keys = list(gj_data.keys())
# Print the list of keys obtained from the 'gj_data' object
print("The loaded JSON dictionary has the following keys:")
print(gj_keys)

The loaded JSON dictionary has the following keys:
['displayFieldName', 'fieldAliases', 'geometryType', 'spatialReference', 'fields', 'features']



## Distance computations with Pyproj

One issue in performing geodetic computation is that any (all) geographic coordinate systems are eventually translated to the surface of the earth - which is not flat. That means every computation of distance between two points is some kind of arc (not actually a straight line). Further the earth is not a true sphere, its a type of ellipsoid. That means the amount of curvature varies depending upon where we are on the surface and the direction - which changes the distance.

Lucky for us there are geographers who like to write code and have built systems to simplify the computation of distances over the earth's surface. One of those systems is called [Pyproj](https://pyproj4.github.io/pyproj/stable/index.html). It has functions that will convert coordinate points between (almost) any two of the many different geographic coordinate systems. As well, Pyproj provides ways to compute distances between two points (mostly assuming the points are already in the same coordinate system).

This example uses the Geod() object to calculate the distance between a slected starting city and all of the cities defined in our CITY_LOCATIONS dictionary (see CONSTANTS above).

The example calls the distances computed 'straight line' distances - because that is what we would have to use to find the distance between two cities using Google. If we didn't use some form of language like that Google would map roads to get us between a source and destination; that would never match our calculation.


## Convert points between geodetic coordinate systems

One of the constraints in doing geodetic computations is that most of the time we need to have our points (the coordinates for places) in the same geographic coordinate system. There are tons and tons of coordinate systems. We can find descriptions of many of them at [EPSG.io](https://epsg.io).

Looking at the wildfire header information, we can see fields named "geometryType" and "spatialReference". This looks like:

        "geometryType": "esriGeometryPolygon",
        "spatialReference": {
            "wkid": 102008,
            "latestWkid": 102008
        },

This indicates that the geometry of our wildfire data are generic polygons and that they are expressed in a coordinate system with the well-known ID (WKID) 102008. This coordinate system is also known as [ESRI:102008](https://epsg.io/102008)

If we look back we might have wondered about the line of code that says:

    geocalc = Geod(ellps='WGS84')         # Use WGS84 ellipsoid representation of the earth

That string, 'WGS84', is a representation of the earth, that also relies on a well known coordinate system that is sometimes called 'decimal degrees' (DD). That decimal degrees system has an official name (or WKID) of [EPSG:4326](https://epsg.io/4326).

For the example below, what we're going to do is take the geometry of a fire feature, extract the largest ring (i.e., the largest boundary of the fire) and convert all of the points in that ring from the ESRI:102008 coordinate system to EPSG:4326 coordinates.

In [5]:
#
#    Transform feature geometry data
#
#    The function takes one parameter, a list of ESRI:102008 coordinates that will be transformed to EPSG:4326
#    The function returns a list of coordinates in EPSG:4326
def convert_ring_to_epsg4326(ring_data=None):
    converted_ring = list()
    # We use a pyproj transformer that converts from ESRI:102008 to EPSG:4326 to transform the list of coordinates
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    # We'll run through the list transforming each ESRI:102008 x,y coordinate into a decimal degree lat,lon
    for coord in ring_data:
        lat,lon = to_epsg4326.transform(coord[0],coord[1])
        new_coord = lat,lon
        converted_ring.append(new_coord)
    return converted_ring

## Compute distance between a place and a wildfire

The basic problem is knowing how far away a fire is from some location (like a city). One issue is that fires are irregularly shaped so the actual answer to that is a bit dependent upon the exact shape and how we want to think about the notion of 'distance'. For example, should we just find the closest point on the perimiter of a fire and call that the distance? Maybe we should find the centroid of the region, identify that as a geolocation (coordinate) and then calculate the distance to that? We can come up with numerous other ways.

The first bit of code finds the point on the perimiter with the shortest distance to the city (place) and returns the distance as well as the lat,lon of the perimeter point.

The second bit of code calculates the average distance of all perimeter points to the city (place) and returns that average as the distance. This is not quite what the centroid would be, but it is probably fairly close.

These are two reasonable ways to think about possible distance to a fire. But both require computing distance to a whole set of points on the perimeter of a fire.


In [6]:
#    
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns a list containing the shortest distance to the perimeter and the point where that is
#
def shortest_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    closest_point = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        # convert the distance to miles
        distance_in_miles = d[2]*0.00062137
        # if it's closer to the city than the point we have, save it
        if not closest_point:
            closest_point.append(distance_in_miles)
            closest_point.append(point)
        elif closest_point and closest_point[0]>distance_in_miles:
            closest_point = list()
            closest_point.append(distance_in_miles)
            closest_point.append(point)
    return closest_point

In [7]:
# Get a city from our CITY_LOCATIONS constant as our starting position
place = CITY_LOCATION["tulare"]
attributes_list = []
count_curveRings = 0

for feature in tqdm(gj_data['features']):
    try:
        wf_year = feature['attributes']['Fire_Year']
        if 1963 <= wf_year <= 2023:
            ring_data = feature['geometry']['rings'][0]
        
        #   Compute using the shortest distance to any point on the perimeter
            distance = shortest_distance_from_place_to_fire_perimeter(place['latlon'],ring_data)

            if distance[0] <= 1250.00:
                feature_attributes = feature['attributes']
                feature_attributes['Distance'] = distance[0]
                attributes_list.append(feature_attributes)
    except Exception as e:
        count_curveRings += 1

print(f"Number of curveRings : {count_curveRings}")


# Create a DataFrame from the list of feature dictionaries
df = pd.DataFrame(attributes_list)


100%|████████████████████████████████████████████████████████████████████████| 135061/135061 [5:20:36<00:00,  7.02it/s]


Number of curveRings : 35


#### There are 35 instances of curveRings out of the 135061 instances, so we can choose to ignore them without much impact on the overall result.

In [8]:
# Save the entire dataframe with the fires filtered based on distance and year, to save time
df.to_csv('../intermediate data/all_fire_data_with_distance.csv', index=False)